## **Use Case Overview**

Imagine you’re part of an R&D team that needs to merge structured data (e.g., experimental results, market trends) from Microsoft Fabric with unstructured documents (e.g., research reports, engineering notes) in SharePoint, then validate these findings against external references (e.g., Bing) and a high-quality “ground truth” internal knowledge stores (e.g., Azure AI Search).

This was a classic Retrieval-Augmented Generation (RAG) scenario—multiple data sources must be queried in real time and cross-checked for consistency. However, by leveraging Azure AI agent services (an agetic enterprise-ready microservices approach) alongside frameworks like Semantic Kernel, we can evolve beyond basic RAG into a mostly autonomous, agentic system. In this design, Agentic RAG and the Reflection Pattern enable each agent to iteratively refine its output until it’s confident in delivering a high-quality, validated answer—paving the way for intelligent automation that continually learns and improves.

To summarize, you’re not only bringing data to the AI but also bringing AI to the data, thus maximizing the value of your knowledge stores. By leveraging state-of-the-art retrieval solutions like Azure AI Search, while also tapping sources such as SharePoint (unstructured data) and Fabric (structured data), you can harness your most valuable asset—data—to achieve new levels of insight and automation. 

**In this demo, we have two Azure AI Agents (extending beyond a single-agent architecture):**

+ DataRetrievalAgent: Has access to Microsoft Fabric (for structured data) and SharePoint (for unstructured documents). Its job is to gather relevant internal data: for example, “failure rates of Material X in high-temperature tests,” or “engineering notes on prior tests.”

- ValidationInsightsAgent Has access to Bing / Azure Cognitive Search for external references and can run a “reflection” or “validation” step. Its job is to cross-check what was returned by the first agent and highlight missing or conflicting information. ValidationInsightsAgent has access to highly curated knowledge sources (e.g., Azure AI Search) for validating the accuracy or truthfulness of the information it receives from DataRetrievalAgent.


**Moving from a single-agent setup to a multi-agent system is now simpler than ever with Semantic Kernel. The general flow looks like this:**

1. The user asks a question (e.g., “Retrieve historical failure rates for Material X in extreme temperatures and cross-check if new standards or conflicting data exist.”).
2. The DataRetrievalAgent fetches structured data from Fabric (e.g., lab test results, analytics) and unstructured docs from SharePoint (e.g., research memos, engineering notes).
3. The ValidationInsightsAgent then queries Bing/Azure Search to verify or supplement the results. Employ a reflection pattern, where it iterates over the combined results, looking for gaps or inconsistencies. If needed, it loops back to the DataRetrievalAgent for clarifications or additional data.

Finally, the user receives a validated, summarized answer that merges internal data with external cross-checks. Thanks to the agents’ back-and-forth reflection.

### **Why This Matters**

+ **Reduced Manual Research**: Instead of manually sifting through multiple data silos and external search engines, the AI Agents automate data gathering and vetting.
+ **Higher Confidence**: Validation ensures data accuracy and highlights missing pieces, improving R&D decision-making.
+ **Enterprise-Grade Security**: Each agent can enforce On-Behalf-Of (OBO) authentication to protect sensitive data (e.g., only pulling data the user is authorized to see).
In the Jupyter Notebook

When you run the code in this Jupyter notebook:

(TODO)

In [1]:
import os
import re
import time
import logging
import json
from datetime import datetime as pydatetime
from typing import Any, List, Dict, Optional
from dotenv import load_dotenv
import asyncio
from datetime import timedelta

# Azure AI Projects
from azure.identity.aio import DefaultAzureCredential
from azure.core.exceptions import HttpResponseError

# semantic kernel
from semantic_kernel.contents import AuthorRole
from semantic_kernel.agents.azure_ai import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.open_ai.run_polling_options import RunPollingOptions

# Load environment variables from .env file
load_dotenv()

# configure logging
from utils.ml_logging import get_logger

logger = get_logger()

# set the directory to the location of the script
try:
    target_directory = os.getenv("TARGET_DIRECTORY", os.getcwd())  # Use environment variable if available
    if os.path.exists(target_directory):
        os.chdir(target_directory)
        logging.info(f"Successfully changed directory to: {os.getcwd()}")
    else:
        logging.error(f"Directory does not exist: {target_directory}")
except Exception as e:
    logging.exception(f"An error occurred while changing directory: {e}")

### **Create Client and Load Azure AI Foundry**

Here, we initialize the Azure AI client using DefaultAzureCredential. This allows us to authenticate and connect to the Azure AI service.


In [2]:
project_client = AzureAIAgent.create_client(credential=DefaultAzureCredential())

### **1. Creating Azure AI Agents: DataRetrievalAgent**

The DataRetrievalAgent is responsible for internal data retrieval, combining structured data from Microsoft Fabric with unstructured documents from SharePoint. This agent ensures that research teams can efficiently access critical R&D insights, such as historical failure rates, experimental results, and engineering notes—all while maintaining secure and authorized access controls.

Agent Capabilities
+ ✅ Structured Data Retrieval → Queries Microsoft Fabric for experiment logs, test results, and structured analytics.
+ ✅ Unstructured Document Search → Fetches relevant reports, blueprints, and research notes from SharePoint.
+ ✅ OBO Authentication → Uses On-Behalf-Of (OBO) authentication to ensure users can only access data they are permitted to view.

For a detailed breakdown of how to create a single Azure AI Agent and configure its data (tools) connections, please refer to:
📌 [01-single-agents-with-azure-ai-agents.ipynb](01-single-agents-with-azure-ai-agents.ipynb).

In [8]:
from azure.core.exceptions import ServiceRequestError
from azure.ai.projects.aio import AIProjectClient

async def get_connection_id(client: AIProjectClient, env_var: str) -> Optional[str]:
    """
    Retrieves the connection object using a connection name stored in an environment variable.

    Args:
        client: The Azure AI Project client.
        env_var (str): The environment variable holding the connection name.

    Returns:
        Connection object if found, otherwise raises an error.
    """
    connection_name = os.getenv(env_var)
    if not connection_name:
        logger.error(f"Missing environment variable: '{env_var}'")
        raise ValueError(f"Environment variable '{env_var}' is required.")

    try:
        connection = await client.connections.get(connection_name=connection_name)
        logger.info(f"Retrieved Connection ID for {env_var}: {connection.id}")
        return connection
    except Exception as e:
        logger.error(f"Failed to retrieve connection for {env_var}: {e}")
        raise

In [9]:
from azure.ai.projects.models import (
    SharepointTool,
    FabricTool,
    ToolSet,
)

# Initialize Azure AI Agent settings
dataretrievalagent_settings = AzureAIAgentSettings.create()

toolset = ToolSet()

try:
    # Retrieve and add SharePoint Tool
    sharepoint_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_SHAREPOINT")
    toolset.add(SharepointTool(connection_id=sharepoint_connection.id))

    # # Retrieve and add Fabric Tool
    # fabric_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_FABRIC")
    # toolset.add(FabricTool(connection_id=fabric_connection.id))

    # logger.info("Successfully created ToolSet with SharePoint and Fabric tools.")
except Exception as e:
    logger.error(f"Failed to create ToolSet: {e}")
    raise

2025-03-20 09:39:27,626 - micro - MainProcess - INFO     Retrieved Connection ID for TOOL_CONNECTION_NAME_SHAREPOINT: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-zhuoqunliai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-1959/connections/ContosoAgentDemoSharepoint (2375541312.py:get_connection_id:22)
INFO:micro:Retrieved Connection ID for TOOL_CONNECTION_NAME_SHAREPOINT: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-zhuoqunliai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-1959/connections/ContosoAgentDemoSharepoint


In [10]:
dataretrievalagent_settings_definition = await project_client.agents.create_agent(
    model=dataretrievalagent_settings.model_deployment_name,
    name="DataRetrievalAgent",
    description=(
        "An AI agent designed to retrieve and integrate structured data from Microsoft Fabric "
        "and unstructured documents from SharePoint to provide comprehensive R&D insights."
    ),
    instructions=(
        "You are an AI assistant specialized in retrieving data from internal enterprise sources. "
        "Utilize the provided tools to access and integrate information from Microsoft Fabric and SharePoint. "
        "Ensure that all responses are based on the most relevant and recent data available."
    ),
    toolset=toolset,
    headers={"x-ms-enable-preview": "true"},
    temperature=0.7,
    top_p=1,
    metadata={
        "use_case": "Internal Data Retrieval for R&D",
        "data_source": "Microsoft Fabric and SharePoint",
        "response_validation": "Ensure data accuracy and relevance"
    },
)

# Print the agent's run ID (agent ID)
print(f"DataRetrievalAgent Run ID: {dataretrievalagent_settings_definition.id}")

DataRetrievalAgent Run ID: asst_hxkuQFK54htjhyCOO1hmbYPB


In [11]:
dataretrievalagent = AzureAIAgent(
    client=project_client,
    definition=dataretrievalagent_settings_definition,
    polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
)

thread = await project_client.agents.create_thread()

USER_INPUTS = [
    "What are the latest trends in R&D?",
    "What are the key features from Dexcom G7 CGM System?",
    "What documents are available in SharePoint related to R&D?",
    "How does Product A compare to Product B in terms of MARD percentage across different glucose ranges?"]
try:
    for user_input in USER_INPUTS:
        # Add the user input as a chat message
        await dataretrievalagent.add_chat_message(thread_id=thread.id, message=user_input)
        print(f"👤 **User:** {user_input}\n")
        
        # Invoke the agent for the specified thread and stream the response
        async for content in dataretrievalagent.invoke(thread_id=thread.id):
            # Only print non-tool messages
            if content.role != AuthorRole.TOOL:
                print(f"🤖 **Agent:** {content.content}\n")
                
except HttpResponseError as e:
    try:
        error_json = json.loads(e.response.content)
        logging.error(f"❌ **Error Message:** {error_json.get('Message')}")
    except json.JSONDecodeError:
        logging.error(f"❌ **Non-JSON Error Content:** {e.response.content}")

👤 **User:** What are the latest trends in R&D?

🤖 **Agent:** The latest trends in research and development (R&D) for 2023 encompass several areas, with significant advancements noted in various fields:

1. **Diabetes Technology**:
   - **Automated Insulin Delivery Systems**: The approval and implementation of systems like the Omnipod 5, which integrates a tubeless pod insulin pump with a continuous glucose monitoring sensor, allowing for automated insulin delivery based on real-time glucose levels.
   - **Advanced Hybrid Closed-Loop Systems**: Devices such as the MiniMed 780G, which offer more precise glycemic control using advanced algorithms.
   - **Bionic Pancreas**: Innovations like the bionic pancreas, which have shown promising results in managing type 1 diabetes through randomized trials.

These advancements highlight a trend towards integrating technology and healthcare, particularly through devices that offer automated and real-time health management solutions【7:1†source】【5:1†

### **2. Creating Azure AI Agents: ValidationInsightsAgent**

The ValidationInsightsAgent is designed to validate and cross-check the data retrieved by the DataRetrievalAgent. It accesses external references such as Bing and Azure Cognitive Search to verify and supplement the internal data. Additionally, it leverages highly curated knowledge sources (e.g., Azure AI Search) to ensure the accuracy and truthfulness of the information, using a reflection or validation step to highlight missing or conflicting details.

Agent Capabilities:

+ ✅ External Reference Verification → Queries Bing and Azure Cognitive Search for real-time validation.
+ ✅ Reflection & Validation Step → Iteratively reviews and refines the information received from the DataRetrievalAgent.
+ ✅ Curated Knowledge Validation → Uses Azure AI Search to confirm the accuracy and reliability of internal data.

For a detailed breakdown of how to create a ValidationInsightsAgent and configure its external tools and connections, please refer to:
📌 [01-single-agents-with-azure-ai-agents.ipynb](01-single-agents-with-azure-ai-agents.ipynb).


In [5]:
from azure.ai.projects.models import (
    BingGroundingTool,
    AzureAISearchTool,
    FileSearchTool,
    VectorStore,
    OpenAIFile
)

# Initialize Azure AI Agent settings
validationinsightagent_settings = AzureAIAgentSettings.create()

# Create a ToolSet to manage tools
toolset = ToolSet()

try:
    # Retrieve and add the Bing Grounding Tool
    bing_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_BING")
    toolset.add(BingGroundingTool(connection_id=bing_connection.id))
    logger.info("Bing Grounding Tool added successfully.")

    # Retrieve and add the Azure AI Search Tool
    # search_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_SEARCH")
    # azure_ai_search_connection = AzureAISearchTool(
    #     index_connection_id=search_connection.id,
    #     index_name="ai-agentic-index"
    # )
    # toolset.add(azure_ai_search_connection)
    # logger.info("Azure AI Search Tool added successfully.")

    pdf_file_path = r"C:\Users\pablosal\Desktop\azure-ai-agent-services-demo\data\product_data\ProductATechncialArchitecture.pdf"
    logger.info(f"Using PDF file path: {pdf_file_path}")

    file: OpenAIFile = await project_client.agents.upload_file_and_poll(file_path=pdf_file_path, purpose="assistants")
    vector_store: VectorStore = await project_client.agents.create_vector_store_and_poll(
        file_ids=[file.id], name="my_vectorstore"
    )

    # 2. Create file search tool with uploaded resources
    file_search = FileSearchTool(vector_store_ids=[vector_store.id])

    toolset.add(file_search)
    logger.info("Azure AI Search Tool added successfully.")
 

    logger.info("Successfully created ToolSet with Bing and File Search tools.")
except Exception as e:
    logger.error(f"Failed to create ToolSet: {e}")
    raise

2025-03-20 08:21:38,847 - micro - MainProcess - INFO     Retrieved Connection ID for TOOL_CONNECTION_NAME_BING: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-zhuoqunliai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-1959/connections/agentsbinggrounding (2375541312.py:get_connection_id:22)
INFO:micro:Retrieved Connection ID for TOOL_CONNECTION_NAME_BING: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-zhuoqunliai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-1959/connections/agentsbinggrounding
2025-03-20 08:21:38,851 - micro - MainProcess - INFO     Bing Grounding Tool added successfully. (3079022251.py:<module>:19)
INFO:micro:Bing Grounding Tool added successfully.
2025-03-20 08:21:38,856 - micro - MainProcess - INFO     Using PDF file path: C:\Users\pablosal\Desktop\azure-ai-agent-services-demo\data\product_data\ProductATechncialArchitecture.pdf (3079022251.py:<module>:31)
INFO:micro:Using PDF file pa

In [11]:
# Create or update a new Validation Insights Agent
validationinsightsagent_definition = await project_client.agents.create_agent(
    model=dataretrievalagent_settings.model_deployment_name,
    name="ValidationInsightsAgent",
    description=(
        "An AI agent designed to validate and refine R&D insights by cross-checking "
        "both internal enterprise data sources (e.g., file search vector store, SharePoint, Fabric) "
        "and external public data (Bing, Azure AI Search). It uses a reflection pattern "
        "to ensure response accuracy, consistency, and proper citations."
    ),
    instructions=(
        "You are a 'Validation Insights' AI assistant, specialized in combining internal enterprise data "
        "(from file search vector stores) and with external search capabilities "
        "(Bing and Azure AI Search). "
        "\n\n"
        "1. **Data Retrieval & Cross-Validation**: Always consult the internal file search index first. "
        "If internal data is insufficient or needs verification, leverage Bing and File Store to find "
        "additional context and resolve discrepancies. "
        "\n\n"
        "2. **Reflection Step**: After retrieving data, pause and reflect on the consistency and relevance "
        "of the combined information. If you find contradictions or gaps, perform a second pass to refine "
        "and unify the insights. "
        "\n\n"
        "3. **Accuracy & Citations**: In your final response, prioritize clarity, correctness, and completeness. "
        "Include citations (links or references) to the sources (internal or external) you used, ensuring "
        "the user understands where information came from. "
        "\n\n"
        "4. **User-Focused Delivery**: Present your findings in a concise, professional manner. If certain "
        "details are not confirmed or require further validation, be transparent. "
        "\n\n"
        "Aim to deliver responses that combine the best aspects of internal knowledge with the breadth "
        "of external web data, ensuring your insights are well-supported and actionable."
    ),
    toolset=toolset,
    # Prefer the internal file search vector store by default
    tool_resources=file_search.resources,
    headers={"x-ms-enable-preview": "true"},
    temperature=0.7,
    top_p=1,
    metadata={
        "use_case": "Cross-Validation and Insight Generation for R&D",
        "data_source": "Internal (Fabric, SharePoint) and External (Bing, Azure AI Search)",
        "response_validation": (
            "Employ a reflection step to cross-check data accuracy "
            "and provide clear citations in final responses."
        )
    },
)

print(f"ValidationInsightsAgent Run ID: {validationinsightsagent_definition.id}")


ValidationInsightsAgent Run ID: asst_kdFT72VdYH0YpoG3tJ5lmoFy


In [12]:
validation_agent = AzureAIAgent(
    client=project_client,
    definition=validationinsightsagent_definition,
    polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
)

# Create a conversation thread for the agent
thread = await project_client.agents.create_thread()

# Define a list of user inputs designed to trigger both internal search (e.g., product architecture) 
# and external market trend validation.
USER_INPUTS = [
    "What are the characteristics and architecture of Product A?",
    "What are the key features from Dexcom G7 CGM System?",
]

try:
    for user_input in USER_INPUTS:
        # Add the user input as a chat message
        await validation_agent.add_chat_message(thread_id=thread.id, message=user_input)
        print(f"👤 **User:** {user_input}\n")
        
        # Invoke the agent for the specified thread and stream the response
        async for content in validation_agent.invoke(thread_id=thread.id):
            # Only print non-tool messages
            if content.role != AuthorRole.TOOL:
                print(f"🤖 **Agent:** {content.content}\n")
                
except HttpResponseError as e:
    try:
        error_json = json.loads(e.response.content)
        logger.error(f"❌ **Error Message:** {error_json.get('Message')}")
    except json.JSONDecodeError:
        logger.error(f"❌ **Non-JSON Error Content:** {e.response.content}")

👤 **User:** What are the characteristics and architecture of Product A?

🤖 **Agent:** ### Characteristics and Architecture of Product A

#### Characteristics
Product A is an advanced integrated continuous glucose monitoring (iCGM) system designed by Contoso Enterprise. It features several state-of-the-art components and functionalities that ensure high accuracy, user comfort, and robust data management. Key characteristics include:

1. **Sensor Module**:
   - **Sensor Core**: Factory-calibrated electrochemical sensor measuring interstitial glucose every 5 minutes, with an enzyme-coated electrode for high sensitivity across a glucose range of 40–400 mg/dL.
   - **Protective Membrane**: Micro-porous, biocompatible membrane that minimizes biofouling and enhances signal stability.
   - **Adhesive and Patch**: Integrated within a hypoallergenic adhesive patch suitable for long-term wear (up to 10 days), with an overpatch for additional durability during physical activities【4:0†source】.

2. 

### **2. Creating Multi Agent System**

The following sample demonstrates how to create an OpenAI assistant using either Azure OpenAI or OpenAI, a chat completion agent and have them participate in a group chat to work towards the user's requirement.


In [3]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

In [4]:
import os
import asyncio
import json
from typing import List, Dict, Optional

from azure.identity.aio import DefaultAzureCredential
from azure.core.exceptions import HttpResponseError
from semantic_kernel import Kernel
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents.azure_ai import AzureAIAgent
from semantic_kernel.agents.strategies import TerminationStrategy, KernelFunctionSelectionStrategy
from semantic_kernel.functions import KernelFunctionFromPrompt


##############################################################################
# 2) A Custom Termination Strategy
###############################################################################
class ApprovalTerminationStrategy(TerminationStrategy):
    """
    Ends the conversation if the Evaluator agent's last output includes the word 'approved'.
    """
    def __init__(self, agents, maximum_iterations=10):
        super().__init__(maximum_iterations=maximum_iterations)
        self.agents = agents

    async def should_agent_terminate(self, agent, history) -> bool:
        # We assume the Evaluator is among self.agents
        # If the final content from the Evaluator includes 'approved', we end
        last_msg = history[-1]
        return (last_msg.name in [a.name for a in self.agents]) and ("approved" in last_msg.content.lower())

###############################################################################
# 3) Creating a Kernel for the Selection Function
###############################################################################
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    """
    Creates a Semantic Kernel instance with an Azure OpenAI chat completion service.
    Make sure you have environment variables for your Azure OpenAI keys/endpoint.
    """
    from semantic_kernel import Kernel
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    kernel = Kernel()
    # Retrieve environment vars (adjust to your naming)
    AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
    AZURE_OPENAI_API_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
    AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
    AZURE_AOAI_CHAT_MODEL_DEPLOYMENT = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_ID")  

    # Register an Azure Chat Completion service in the kernel
    kernel.add_service(
        service=AzureChatCompletion(
            deployment_name=AZURE_AOAI_CHAT_MODEL_DEPLOYMENT,
            api_key=AZURE_OPENAI_KEY,
            endpoint=AZURE_OPENAI_API_ENDPOINT,
            api_version=AZURE_OPENAI_API_VERSION,
        )
    )
    return kernel

###############################################################################
# 4) Example LLM Prompt for Multi-Agent Turn Selection
###############################################################################
RETRIEVER_NAME = "ValidationInsightsAgent"
EVALUATOR_NAME = "DataRetrievalAgent"

selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt=f"""
Determine which participant takes the next turn in a conversation based on the most recent participant.
State only the name of the participant to take the next turn.
No participant should take more than one turn in a row.

Choose only from these participants:
- {RETRIEVER_NAME}
- {EVALUATOR_NAME}

Always follow these rules when selecting the next participant:
- {RETRIEVER_NAME} retrieving the document or relevant data the query.
- After {RETRIEVER_NAME}, it is {EVALUATOR_NAME}'s turn to evaluate the content.
- After {EVALUATOR_NAME}, the workflow may terminate if 'approved'.

History:
{{{{$history}}}}
""",
)

###############################################################################
# 5) Utility Logging For Agent Interaction
###############################################################################
def log_agent_invocation(agent_name: str, role: str, input_data: str, output_data: str):
    trace_data = {
        "agent_name": agent_name,
        "role": role,
        "input": input_data,
        "output": output_data,
    }
    logger.info("AGENT INVOCATION:\n" + json.dumps(trace_data, indent=4))

###############################################################################
# 6) Main Async Function
###############################################################################
async def main():
    # 2. Initialize the credential & AzureAIAgent client
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        # Pre-existing Azure AI Agents (IDs from your environment)
        ValidationInsightsAgentID = "asst_kdFT72VdYH0YpoG3tJ5lmoFy"
        DataRetrievalAgentID = "asst_Wo0GJ9MpmvkfRPNwllC7bYFS"

        # 3. Retrieve each agent definition from your Azure AI Agent Service
        #    Assume the Formulator & Evaluator are the same agent or different? 
        #    Below, just demonstrating you might have 2 definitions for 3 roles.
        dataretrieval_def = await client.agents.get_agent(agent_id=DataRetrievalAgentID)
        validation_def = await client.agents.get_agent(agent_id=ValidationInsightsAgentID)

        # 4. Build SK Agents from these definitions
        agent_retriever = AzureAIAgent(client=client, definition=dataretrieval_def)
        agent_evaluator = AzureAIAgent(client=client, definition=validation_def)

        # 5. Setup the multi-agent chat
        chat = AgentGroupChat(
            agents=[agent_retriever, agent_evaluator],
            termination_strategy=ApprovalTerminationStrategy(
                maximum_iterations=10,
                agents=[agent_evaluator],  # The Evaluator decides final approval
            ),
            selection_strategy=KernelFunctionSelectionStrategy(
                function=selection_function,
                kernel=_create_kernel_with_chat_completion("selection"),
                # We parse the output of the LLM prompt to choose next agent
                result_parser=lambda result: str(result.value[0]) if result.value else EVALUATOR_NAME,
                agent_variable_name="agents",   # the variable in the prompt
                history_variable_name="history", # the variable in the prompt
            ),
        )

        # Example user system or user message
        system_message = """
        You are orchestrating a multi-step R&D conversation between two roles: 
        1) Retriever – gathers internal data from Fabric/SharePoint and external references from Bing/Azure Search.
        2) Evaluator – cross-validates the retrieved data, identifies missing insights, and decides if it is 'approved' or if more data is needed.

        **Demo Flow: AI-Powered R&D and Product Design**
        1. A researcher initiates a query for internal experiment data and external market trends to refine a new product design.
        2. Retriever queries Fabric AI Skill for structured results (e.g., experiment data), and SharePoint for related research papers, engineering reports, and design notes.
        3. Evaluator checks the alignment between internal test results and external market or regulatory feedback. Identifies any conflicting standards or data gaps. If needed, it reruns queries for additional sources.
        4. The system compiles an AI-generated report summarizing key findings, highlighting risk factors, data gaps, and actionable recommendations.
        5. The researcher can ask follow-up questions about materials, regulatory risks, or design improvements. 

        The user can keep asking questions until the Evaluator decides it is 'approved' (final answer) or more data is required.
        """
        await chat.add_chat_message(message=system_message)
        logger.info(f"System Setup: {system_message}")

        # Updated user message introducing the high-level request
        user_message = """
        What are the characteristics and architecture of Product A? Please research the market for similar products and gather insights from our research team. What other products have we done competitor analysis on?
        """
        await chat.add_chat_message(message=user_message)
        logger.info(f"User Input: {user_message}")

        # 6. Real-time streaming of conversation
        try:
            last_input = user_message
            async for content in chat.invoke():
                agent_name = content.name or "Unknown"
                agent_role = content.role.name
                agent_output = content.content

                # Log invocation
                log_agent_invocation(agent_name, agent_role, last_input, agent_output)

                # Print
                print(f"{agent_role} - {agent_name}: {agent_output}")

                last_input = agent_output

            logger.info(f"Workflow Complete? {chat.is_complete}")

        except HttpResponseError as err:
            logger.error(f"Error while streaming conversation: {err}")

        finally:
            # 7. Optionally reset or cleanup if ephemeral
            await chat.reset()
            # If you'd like to delete ephemeral agent definitions, do so here
            # e.g., await client.agents.delete_agent(DataRetrievalAgentID)
            #       await client.agents.delete_agent(ValidationInsightsAgentID)



In [5]:
await main() 

2025-03-20 11:45:19,948 - micro - MainProcess - INFO     System Setup: 
        You are orchestrating a multi-step R&D conversation between two roles: 
        1) Retriever – gathers internal data from Fabric/SharePoint and external references from Bing/Azure Search.
        2) Evaluator – cross-validates the retrieved data, identifies missing insights, and decides if it is 'approved' or if more data is needed.

        **Demo Flow: AI-Powered R&D and Product Design**
        1. A researcher initiates a query for internal experiment data and external market trends to refine a new product design.
        2. Retriever queries Fabric AI Skill for structured results (e.g., experiment data), and SharePoint for related research papers, engineering reports, and design notes.
        3. Evaluator checks the alignment between internal test results and external market or regulatory feedback. Identifies any conflicting standards or data gaps. If needed, it reruns queries for additional sources.
 

ASSISTANT - ValidationInsightsAgent: ### Product A: Characteristics and Architecture

**Product A** is an advanced integrated continuous glucose monitoring (iCGM) system developed by Contoso Enterprise. Below is a summary of its key characteristics and architecture:

#### **1. Sensor Module**
- **Design & Construction**:
  - **Sensor Core**: Factory-calibrated electrochemical sensor measures interstitial glucose every 5 minutes.
  - **Protective Membrane**: Micro-porous, biocompatible membrane minimizes biofouling and interference.
  - **Adhesive and Patch**: Hypoallergenic adhesive patch supports long-term wear (up to 10 days) with an overpatch for additional durability during physical activities.

#### **2. Transmitter and Communication Layer**
- **Low-Power Transmitter**: Converts analog sensor signals to digital data every 5 minutes, with a battery life supporting a full sensor session and a 12-hour grace period.
- **Data Connectivity**:
  - **Wireless Transmission**: Secure Blueto

2025-03-20 11:46:12,047 - micro - MainProcess - INFO     AGENT INVOCATION:
{
    "agent_name": "ValidationInsightsAgent",
    "role": "ASSISTANT",
    "input": "### Product A: Characteristics and Architecture\n\n**Product A** is an advanced integrated continuous glucose monitoring (iCGM) system developed by Contoso Enterprise. Below is a summary of its key characteristics and architecture:\n\n#### **1. Sensor Module**\n- **Design & Construction**:\n  - **Sensor Core**: Factory-calibrated electrochemical sensor measures interstitial glucose every 5 minutes.\n  - **Protective Membrane**: Micro-porous, biocompatible membrane minimizes biofouling and interference.\n  - **Adhesive and Patch**: Hypoallergenic adhesive patch supports long-term wear (up to 10 days) with an overpatch for additional durability during physical activities.\n\n#### **2. Transmitter and Communication Layer**\n- **Low-Power Transmitter**: Converts analog sensor signals to digital data every 5 minutes, with a battery 

ASSISTANT - ValidationInsightsAgent: ### Product A: Characteristics and Architecture

**Product A** is an advanced integrated continuous glucose monitoring (iCGM) system developed by Contoso Enterprise. Below is a summary of its key characteristics and architecture:

#### **1. Sensor Module**
- **Design & Construction**:
  - **Sensor Core**: Factory-calibrated electrochemical sensor measures interstitial glucose every 5 minutes.
  - **Protective Membrane**: Micro-porous, biocompatible membrane minimizes biofouling and interference.
  - **Adhesive and Patch**: Hypoallergenic adhesive patch supports long-term wear (up to 10 days) with an overpatch for additional durability during physical activities.

#### **2. Transmitter and Communication Layer**
- **Low-Power Transmitter**: Converts analog sensor signals to digital data every 5 minutes, with a battery life supporting a full sensor session and a 12-hour grace period.
- **Data Connectivity**:
  - **Wireless Transmission**: Secure Blueto

AgentInvokeException: Run failed with status: `RunStatus.FAILED` for agent `DataRetrievalAgent` and thread `thread_eT2qkRdpTxTq0n1z2YYA23JP` with error: Error: sharepoint_tool_server_error; Internal Server Error

## Creting Streamlit App 

In [ ]:
# 2. Import Necessary Modules
# Import the required modules in your Python script:

import os
import asyncio
import streamlit as st
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel import Kernel
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents.azure_ai import AzureAIAgent
from semantic_kernel.agents.strategies import TerminationStrategy, KernelFunctionSelectionStrategy
from semantic_kernel.functions import KernelFunctionFromPrompt


In [ ]:
# 3. Define the Approval Termination Strategy
# Create a custom termination strategy to end the conversation based on specific criteria:

class ApprovalTerminationStrategy(TerminationStrategy):
    def __init__(self, agents, maximum_iterations=10):
        super().__init__(maximum_iterations=maximum_iterations)
        self.agents = agents

    async def should_agent_terminate(self, agent, history) -> bool:
        last_msg = history[-1]
        return (last_msg.name in [a.name for a in self.agents]) and ("approved" in last_msg.content.lower())


In [ ]:
# 4. Initialize the Kernel with Chat Completion
# Set up the Semantic Kernel with an Azure OpenAI chat completion service:

def _create_kernel_with_chat_completion() -> Kernel:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    kernel = Kernel()
    AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
    AZURE_OPENAI_API_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
    AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
    AZURE_AOAI_CHAT_MODEL_DEPLOYMENT = os.getenv("AZURE_OPENAI_CHAT_MODEL_DEPLOYMENT", "gpt-4")

    kernel.add_service(
        service=AzureChatCompletion(
            deployment_name=AZURE_AOAI_CHAT_MODEL_DEPLOYMENT,
            api_key=AZURE_OPENAI_KEY,
            endpoint=AZURE_OPENAI_API_ENDPOINT,
            api_version=AZURE_OPENAI_API_VERSION,
        )
    )
    return kernel


In [ ]:
# 5. Define the Selection Function Prompt
# Create a prompt to determine the next participant in the conversation:

RETRIEVER_NAME = "ValidationInsightsAgent"
EVALUATOR_NAME = "DataRetrievalAgent"

selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt=f"""
Determine which participant takes the next turn in a conversation based on the most recent participant.
State only the name of the participant to take the next turn.
No participant should take more than one turn in a row.

Choose only from these participants:
- {RETRIEVER_NAME}
- {EVALUATOR_NAME}

Always follow these rules when selecting the next participant:
- {RETRIEVER_NAME} retrieves the document or relevant data for the query.
- After {RETRIEVER_NAME}, it is {EVALUATOR_NAME}'s turn to evaluate the content.
- After {EVALUATOR_NAME}, the workflow may terminate if 'approved'.

History:
{{{{$history}}}}
""",
)


In [ ]:
# 6. Initialize Agents and Chat
# Set up the agents and the chat interface:

async def initialize_agents():
    creds = DefaultAzureCredential()
    client = await AzureAIAgent.create_client(credential=creds)

    ValidationInsightsAgentID = "asst_kdFT72VdYH0YpoG3tJ5lmoFy"
    DataRetrievalAgentID = "asst_Wo0GJ9MpmvkfRPNwllC7bYFS"

    dataretrieval_def = await client.agents.get_agent(agent_id=DataRetrievalAgentID)
    validation_def = await client.agents.get_agent(agent_id=ValidationInsightsAgentID)

    agent_retriever = AzureAIAgent(client=client, definition=dataretrieval_def)
    agent_evaluator = AzureAIAgent(client=client, definition=validation_def)

    chat = AgentGroupChat(
        agents=[agent_retriever, agent_evaluator],
        termination_strategy=ApprovalTerminationStrategy(
            maximum_iterations=10,
            agents=[agent_evaluator],
        ),
        selection_strategy=KernelFunctionSelectionStrategy(
            function=selection_function,
            kernel=_create_kernel_with_chat_completion(),
            result_parser=lambda result: str(result.value[0]) if result.value else EVALUATOR_NAME,
            agent_variable_name="agents",
            history_variable_name="history",
        ),
    )

    return chat


In [ ]:
# 7. Build the Streamlit Interface
# Create the Streamlit interface to interact with the agents:

async def main():
    st.title("Multi-Agent Chat Interface")

    if "chat" not in st.session_state:
        st.session_state.chat = await initialize_agents()
        st.session_state.history = []

    user_input = st.chat_input("Enter your message:")
    if user_input:
        st.session_state.history.append({"role": "user", "content": user_input})
        await st.session_state.chat.add_chat_message(message=user_input)

        async for content in st.session_state.chat.invoke():
            agent_name = content.name or "Unknown"
            agent_role = content.role.name
            agent_output = content.content

            st.session_state.history.append({"role": agent_role, "name": agent_name, "content": agent_output})

            st.chat_message(agent_role).write(f"**{agent_name}:** {agent_output}")

        if st.session_state.chat.is_complete:
            st.write("Conversation has been approved and terminated.")
            st.session_state.chat.reset()
            st.session_state.history = []

if __name__ == "__main__":
    asyncio.run(main())


8. Run the Streamlit Application

Save your script (e.g., app.py) and run it using Streamlit:

In [1]:
!streamlit run src/chatapp/app.py


^C
